In [7]:
import numpy as np
import matplotlib as mpl
from pprint import pprint

from matplotlib import pyplot as plt
mpl.use("Agg")
import time

from pprint import pprint

from lib.qvmc.hamiltonian import Hamiltonian
from lib.qvmc.state import State
from lib.qvmc.optimizer import Optimizer
from lib.qvmc.debug import Plotter
from lib.qvmc.utils import get_initial_w

import pickle

from configs import cfg

import os
import datetime

import random

from pdb import set_trace

## 16 Particle Simulation Using Monte Carlo

In [10]:
with open('common_parameters.pickle', 'rb') as fb:
    w_parameters = pickle.load(fb)

initial_parameters = {
                "a" : np.asarray([0 for i in range(cfg['num_particles'])]),
                "b" : np.asarray([0 for i in range(cfg['num_particles'])]) ,
                "w" : w_parameters
                }
        
cfg['fixed_params'] =  {"a" : initial_parameters['a'],
                         "b" : initial_parameters['b']}

plotter = None
delta=0.00
num_steps = 10000

H = Hamiltonian(16, delta=delta)

opt = Optimizer(H , num_steps, cfg=cfg, plotter=plotter,
                        parameters = initial_parameters, lr = 0.02, random_noise_scale=None,
                        fixed_parameters = cfg['fixed_params'])

In [11]:
energy1, x, q = opt.monte_carlo_simulation()
print(f"First Energy: {energy1:.4f}")
f = {}
for key in opt.parameters.keys():
    f[key] = 2*(q[key] - x[key]*energy1) # derivatives vector
x = random.randint(0,15)
y = random.randint(0,15)
print(f"Derivative[{x}][{y}]: {f['w'][x][y]:.4f}")
      
opt.parameters['w'][x][y] = opt.parameters['w'][x][y] + 0.1
energy2, _, _ = opt.monte_carlo_simulation()
print(f"Second energy: {energy2:.4f}")
print(f"Numerical Derivative: {(energy2-energy1)/0.1:.4f}")

First Energy: (-5.967790998845737+0.0012396254187187349j)
Derivative[12][6]: (-0.0024162709011840144+0.011859523517944948j)
Second energy: (-5.991962435028827+0.0016216878534693801j)
Numerical Derivative: (-0.24171436183090478+0.0038206243475064528j)


## 16 Particle Simulation Using Monte Enumeration

In [12]:
from lib.enumeration_qvmc.hamiltonian import Hamiltonian
from lib.enumeration_qvmc.state import State
from lib.enumeration_qvmc.optimizer import Optimizer
from lib.enumeration_qvmc.debug import Plotter
from lib.enumeration_qvmc.utils import get_initial_w

In [25]:
with open('common_parameters.pickle', 'rb') as fb:
    w_parameters = pickle.load(fb)

initial_parameters = {
                "a" : np.asarray([0 for i in range(cfg['num_particles'])]),
                "b" : np.asarray([0 for i in range(cfg['num_particles'])]) ,
                "w" : w_parameters
                }
        
cfg['fixed_params'] =  {"a" : initial_parameters['a'],
                         "b" : initial_parameters['b']}

plotter = None
delta=0.00
#  num_steps = 10000

H = Hamiltonian(16, delta=delta)

opt = Optimizer(H , cfg=cfg, plotter=plotter,
                        parameters = initial_parameters, lr = 0.02, random_noise_scale=None,
                        fixed_parameters = cfg['fixed_params'])

In [26]:
energy1, delta, q = opt.monte_carlo_simulation()
print(f"First Energy: {energy1:.4f}")
f = {}
for key in opt.parameters.keys():
    f[key] = 2*(q[key] - delta[key]*energy1) # derivatives vector
print(f"Derivative[{x}][{y}]: {f['w'][x][y]:.4f}")
      
opt.parameters['w'][x][y] = opt.parameters['w'][x][y] + 0.01
energy2, _, _ = opt.monte_carlo_simulation()
print(f"Second energy: {energy2:.4f}")
print(f"Numerical Derivative: {(energy2-energy1)/0.01:.4f}")

First Energy: (-5.992034655890999+8.106170083000194e-18j)
Derivative[12][6]: (0.003128749213722082-0.004774888176606817j)
Second energy: (-5.991988492802274-1.276524095653105e-17j)
Numerical Derivative: (0.004616308872495978-2.0871411039531244e-15j)
